# app.py

In [ ]:
from flask import Flask, render_template,request
import pickle
import numpy as np
import tensorflow as tf

app = Flask(__name__)


@app.route('/',methods=['GET','POST'])
def index():
    if request.method == 'POST':
            final_data = np.zeros(12)
            final_data[0]=float(request.form['crew'])
            final_data[1] = float(request.form['seat'])
            gsr = pickle.load(open("gsrscalar.sav", "rb"))
            r = pickle.load(open("filtered_rscalar.sav", "rb"))
            ecg = pickle.load(open("filtered_ecgscalar.sav", "rb"))
            eeg_encoder_1 = pickle.load(open("eeg_encoder_1scalar.sav", "rb"))
            eeg_encoder_2 = pickle.load(open("eeg_encoder_2scalar.sav", "rb"))
            eeg_encoder_3 = pickle.load(open("eeg_encoder_3scalar.sav", "rb"))
            eeg_encoder_4 = pickle.load(open("eeg_encoder_4scalar.sav", "rb"))
            eeg_encoder_5 = pickle.load(open("eeg_encoder_5scalar.sav", "rb"))
            component_1 = pickle.load(open("component_1scalar.sav", "rb"))
            component_2 = pickle.load(open("component_2scalar.sav", "rb"))
            gsr = gsr.transform(np.array([float(request.form['gsr'])]).reshape(-1, 1))[0][0]
            r = r.transform(np.array([float(request.form['respiration'])]).reshape(-1, 1))[0][0]
            ecg = ecg.transform(np.array([float(request.form['ecg'])]).reshape(-1, 1))[0][0]
            final_data[2] = gsr  # gsr
            final_data[3] = r  # r
            final_data[4] = ecg  # ecg
            eeg = np.zeros(20)
            eeg[0]=float(request.form['eeg_fp1'])
            eeg[1]=float(request.form['eeg_f7'])
            eeg[2]=float(request.form['eeg_f8'])
            eeg[3]=float(request.form['eeg_t4'])
            eeg[4]=float(request.form['eeg_t6'])
            eeg[5]=float(request.form['eeg_t5'])
            eeg[6]=float(request.form['eeg_t3'])
            eeg[7]=float(request.form['eeg_fp2'])
            eeg[8]=float(request.form['eeg_o1'])
            eeg[9]=float(request.form['eeg_p3'])
            eeg[10]=float(request.form['eeg_pz'])
            eeg[11]=float(request.form['eeg_f3'])
            eeg[12]=float(request.form['eeg_fz'])
            eeg[13]=float(request.form['eeg_f4'])
            eeg[14]=float(request.form['eeg_c4'])
            eeg[15]=float(request.form['eeg_p4'])
            eeg[16]=float(request.form['eeg_poz'])
            eeg[17]=float(request.form['eeg_c3'])
            eeg[18]=float(request.form['eeg_cz'])
            eeg[19]=float(request.form['eeg_o2'])
            feature_encoder = tf.keras.models.load_model('encoder.h5')
            hidden_feature = feature_encoder.predict(eeg.reshape(1,-1))[0]
            eeg_encoder_1 = eeg_encoder_1.transform(hidden_feature[0].reshape(-1, 1))[0][0]
            eeg_encoder_2 = eeg_encoder_2.transform(hidden_feature[1].reshape(-1, 1))[0][0]
            eeg_encoder_3 = eeg_encoder_3.transform(hidden_feature[2].reshape(-1, 1))[0][0]
            eeg_encoder_4 = eeg_encoder_4.transform(hidden_feature[3].reshape(-1, 1))[0][0]
            eeg_encoder_5 = eeg_encoder_5.transform(hidden_feature[4].reshape(-1, 1))[0][0]
            final_data[5] = eeg_encoder_1
            final_data[6] = eeg_encoder_2
            final_data[7] = eeg_encoder_3
            final_data[8] = eeg_encoder_4
            final_data[9] = eeg_encoder_5
            pca_model = pickle.load(open("pca.sav", "rb"))
            component = pca_model.transform(eeg.reshape(1,-1))
            component_1 = component_1.transform(component[0][0].reshape(-1, 1))[0][0]
            component_2 = component_2.transform(component[0][1].reshape(-1, 1))[0][0]
            final_data[10] = component_1
            final_data[11] = component_2
            model = pickle.load(open("lgbt_model.sav", "rb"))
            prediction = model.predict(final_data.reshape(1,-1))
            predict_proba = model.predict_proba(final_data.reshape(1,-1))
            return render_template('results.html',predcited=[prediction,predict_proba])
    else:
        return render_template('index.html')

if __name__ == "__main__":
    app.run()

# index.html

In [ ]:
<!DOCTYPE html>
<html>
<head>
    <meta charset="utf-8">
    <meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no">
    <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/4.0.0/css/bootstrap.min.css" integrity="sha384-Gn5384xqQ1aoWXA+058RXPxPg6fy4IWvTNh0E263XmFcJlSAwiGgFAW/dAiS6JXm" crossorigin="anonymous">
    <title>Reducing Commercial Aviation Fatalities | Kaggle Competition</title>   
</head>
	<style>
    #main-footer
    {
      color: #FFFFFF;
      font-family: "Segoe UI";
      background: #2B2B2B;
      text-align: center;
      margin-top: 125px;
      padding: 16px;
      bottom: 8px;
    }</style>
<body>
<div class="container",class="form-group">
        <h2 >Reducing Commercial Aviation Fatalities | Kaggle Competition</h2>
    <h3 >Enter the Following Details:</h3>

  <form action="/" method="POST">
    <label for="crew">Crew_id : </label>
    <input type="number" step="any", id="crew" name="crew" required>
    <br>

    <label for="seat">Seat (0-left,1-Right) : </label>
    <input type="number" step="any", id="seat" name="seat" required>
    <br>

    <label for="gsr">GSR in microvolts: </label>
    <input type="number" step="any", id="gsr" name="gsr" required>
    <br>

    <label for="ecg">ECG in microvolts: </label>
    <input type="number" step="any", id="ecg" name="ecg" required>
    <br>

    <label for="respiration">Respiration in microvolts: </label>
    <input type="number" step="any", id="respiration" name="respiration" required>
    <br>

    <label for="eeg_fp1"> eeg_fp1: </label>
    <input type="number" step="any", id="eeg_fp1" name="eeg_fp1" required>
    <br>

    <label for="eeg_f7">eeg_f7 : </label>
    <input type="number" step="any", id="eeg_f7" name="eeg_f7" required>
    <br>

    <label for="eeg_f8">eeg_f8: </label>
    <input type="number" step="any", id="eeg_f8" name="eeg_f8" required>
    <br>

    <label for="eeg_t4"> eeg_t4: </label>
    <input type="number" step="any", id="eeg_t4" name="eeg_t4" required>
    <br>

    <label for="eeg_t6"> eeg_t6: </label>
    <input type="number" step="any", id="eeg_t6" name="eeg_t6" required>
    <br>

    <label for="eeg_t5"> eeg_t5: </label>
    <input type="number" step="any", id="eeg_t5" name="eeg_t5" required>
    <br>

    <label for="eeg_t3"> eeg_t3: </label>
    <input type="number" step="any", id="eeg_t3" name="eeg_t3" required>
    <br>

    <label for="eeg_fp2"> eeg_fp2: </label>
    <input type="number" step="any", id="eeg_fp2" name="eeg_fp2" required>
    <br>

    <label for="eeg_o1"> eeg_o1: </label>
    <input type="number" step="any", id="eeg_o1" name="eeg_o1" required>
    <br>

    <label for="eeg_p3"> eeg_p3: </label>
    <input type="number" step="any", id="eeg_p3" name="eeg_p3" required>
    <br>

    <label for="eeg_pz"> eeg_pz: </label>
    <input type="number" step="any", id="eeg_pz" name="eeg_pz" required>
    <br>

    <label for="eeg_f3"> eeg_f3: </label>
    <input type="number" step="any", id="eeg_f3" name="eeg_f3" required>
    <br>

    <label for="eeg_fz"> eeg_fz: </label>
    <input type="number" step="any", id="eeg_fz" name="eeg_fz" required>
    <br>

    <label for="eeg_f4"> eeg_f4 : </label>
    <input type="number" step="any", id="eeg_f4" name="eeg_f4" required>
    <br>

    <label for="eeg_c4"> eeg_c4 : </label>
    <input type="number" step="any", id="eeg_c4" name="eeg_c4" required>
    <br>

    <label for="eeg_p4"> eeg_p4: </label>
    <input type="number" step="any", id="eeg_p4" name="eeg_p4" required>
    <br>

    <label for="eeg_poz"> eeg_poz: </label>
    <input type="number" step="any", id="eeg_poz" name="eeg_poz" required>
    <br>

    <label for="eeg_c3"> eeg_c3:</label>
    <input type="number" step="any", id="eeg_c3" name="eeg_c3" required>
    <br>

    <label for="eeg_cz"> eeg_cz:</label>
    <input type="number" step="any", id="eeg_cz" name="eeg_cz" required>
    <br>

    <label for="eeg_o2"> eeg_o2: </label>
    <input type="number" step="any", id="eeg_o2" name="eeg_o2" required>
    <br>
    <br>

    <input type="submit" value="Predict"><br><br>
  </form>
  <h4>sample_data:</h4>
    <h5>crew:1,seat:1,gsr:388.829987,ecg:-4520.0,respiration:817.705994,
    eeg_fp1:-5.28545,eeg_f7:26.775801,eeg_f8:-9.52731	,eeg_t4:-12.7932,eeg_t6:16.7178
    ,eeg_t5:33.737499,eeg_t3:23.712299,eeg_fp2:-6.69587	,eeg_o1:29.2321,
     eeg_p3:24.842899,eeg_pz:3.92134,eeg_f3:18.447001,eeg_fz:1.07547,eeg_f4:3.09029,eeg_c4:37.368999,
    eeg_p4:17.437599,eeg_poz:19.2019,eeg_c3:20.5968	,eeg_cz:-3.95115,eeg_o2:14.5076</h5>

    </div>


	<footer id="main-footer">
      Copyright &copy; 2020
    </footer>
</body>
</html>

# result.html

In [ ]:


<!DOCTYPE html>
<html>
<head>
    <meta charset="utf-8">
    <meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no">

    <!-- Bootstrap CSS -->
    <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/4.0.0/css/bootstrap.min.css" integrity="sha384-Gn5384xqQ1aoWXA+058RXPxPg6fy4IWvTNh0E263XmFcJlSAwiGgFAW/dAiS6JXm" crossorigin="anonymous">

    <title>Reducing Commercial Aviation Fatalities | Kaggle Competition</title>
    
</head>
	<style>
    #main-footer
    {
      color: #FFFFFF;
      font-family: "Segoe UI";
      background: #2B2B2B;
      text-align: center;
      margin-top: 125px;
      padding: 16px;
      bottom: 8px;
    }</style>
<body>
<div class="container",class="form-group">
        <h1 >Reducing Commercial Aviation Fatalities | Kaggle Competition</h1>
         <h2>states of Event</h2>
         <h6>0 means Baseline</h6>
         <h6>1 means Channelized Attention (CA)</h6>
         <h6>2 means Diverted Attention (DA)</h6>
         <h6>3 means Startle/Surprise (SS)</h6>
        <h2> Predicted state :- {{predcited[0][0]}} </h2>
        <h3> Probablity of state Baseline:- {{predcited[1][0][0]*100}} </h3>
        <h3> Probablity of state Baseline:- {{predcited[1][0][1]*100}} </h3>
         <h3> Probablity of state Baseline:- {{predcited[1][0][2]*100}} </h3>
         <h3> Probablity of state Baseline:- {{predcited[1][0][3]*100}} </h3>

         <a href="/"><h5 >Click here to move back to the home page</h5 ></a>

    </div>
</body>
</html>

# requrements.txt

In [ ]:
gunicorn==20.0.4
Flask==1.1.2
numpy==1.18.5
requests==2.23.0
tensorflow==2.3.1
scikit-learn==0.22.1
lightgbm==3.2.0

# Procfile

In [ ]:
web: gunicorn app:app

# Heroku Deployment Link

https://ancient-eyrie-46230.herokuapp.com/